In [1]:
addprocs(6);

In [2]:
@everywhere begin
    import POMDPModels
    using POMCP
    using POMDPs
    import POMDPToolbox.PreviousObservation

    import POMCP.belief_from_node
    import POMCP.init_V
    import POMCP.POMCPPolicy
    import POMDPs.action
end

In [3]:
N = 2000;
eps = 0.01;
problem = POMDPModels.BabyPOMDP(-5, -10);

In [4]:
@everywhere type RandomBabyPolicy <: POMDPs.Policy
    rng::AbstractRNG
end
@everywhere action(p::RandomBabyPolicy, b::POMDPs.Belief) = POMDPModels.BabyAction(rand(p.rng)>0.5)

In [5]:
@everywhere function belief_from_node(policy::POMCPPolicy, node::POMCP.ObsNode)
    return PreviousObservation(node.label)
end
@everywhere function belief_from_node(policy::POMCPPolicy, node::POMCP.RootNode)
    state = create_state(policy.problem)
    obs = create_observation(policy.problem)
    od = create_observation_distribution(policy.problem)
    rand!(policy.solver.rng, state, node.B)
    observation!(od, policy.problem, state, POMDPModels.BabyAction(false))
    rand!(policy.solver.rng, obs, od)
    return PreviousObservation(obs)
end

In [6]:
function est_reward(problem, policy, belief, N)
    sum = @parallel (+) for i in 1:N
        sim_rng = MersenneTwister(i)
        POMDPs.simulate(problem, policy, belief, rng=sim_rng, eps=eps, initial_state=POMDPModels.BabyState(false))
    end
    return sum/N;
end

est_reward (generic function with 1 method)

In [7]:
# Feed when Crying (Expected Reward for this nearly-optimal policy is -17.14)
# Test from earlier this week with 5000 experiments: -16.72
est_reward(problem, POMDPModels.FeedWhenCrying(), PreviousObservation(POMDPModels.BabyObservation(false)), N)

-16.600315445486913

In [8]:
# Random
pol_rng = MersenneTwister(7)
est_reward(problem, RandomBabyPolicy(pol_rng), PreviousObservation(POMDPModels.BabyObservation(false)), N)

-32.34750118876957

In [ ]:
# POMCP with FWC rollout policy
# test from earlier this week with 5000 experiments: -16.77
rng = MersenneTwister(2)

solver = POMCPSolver(POMDPModels.FeedWhenCrying(),
                     0.01,
                     10,
                     300, 
                     rng,
                     false)

policy = solve(solver, problem)

@time est_reward(problem, policy, POMCPBeliefWrapper(POMDPModels.BabyStateDistribution(0.0)), N)

In [ ]:
# POMCP with Random rollout policy
rng = MersenneTwister(2)
rollout_pol_rng = MersenneTwister(2)

solver = POMCPSolver(RandomBabyPolicy(rollout_pol_rng),
                     0.01,
                     10,
                     300, 
                     rng,
                     false)

policy = solve(solver, problem)

@time est_reward(problem, policy, POMCPBeliefWrapper(POMDPModels.BabyStateDistribution(0.0)), N)